# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 18 2022 8:30AM,244413,10,0085940778,ISDIN Corporation,Released
1,Aug 18 2022 8:30AM,244413,10,0085940782,ISDIN Corporation,Released
2,Aug 18 2022 8:30AM,244413,10,0085940783,ISDIN Corporation,Released
3,Aug 18 2022 8:30AM,244413,10,0085940792,ISDIN Corporation,Released
4,Aug 18 2022 8:30AM,244413,10,0085940822,ISDIN Corporation,Released
5,Aug 18 2022 8:30AM,244425,10,Enova-10495,Emerginnova,Released
6,Aug 18 2022 8:30AM,244425,10,Enova-10496,Emerginnova,Released
7,Aug 18 2022 8:30AM,244425,10,Enova-10498,Emerginnova,Completed
8,Aug 18 2022 8:30AM,244425,10,Enova-10499,Emerginnova,Released
9,Aug 18 2022 8:30AM,244425,10,Enova-10500,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
49,244431,Released,1
50,244432,Released,9
51,244433,Released,2
52,244434,Released,6
53,244435,Released,18


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244431,NaN,NaN,1.0
244432,NaN,NaN,9.0
244433,NaN,NaN,2.0
244434,NaN,NaN,6.0
244435,NaN,NaN,18.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244307,0.0,0.0,5.0
244313,0.0,0.0,2.0
244316,0.0,0.0,1.0
244324,0.0,1.0,0.0
244325,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244307,0,0,5
244313,0,0,2
244316,0,0,1
244324,0,1,0
244325,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244307,0,0,5
1,244313,0,0,2
2,244316,0,0,1
3,244324,0,1,0
4,244325,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244307,,,5
1,244313,,,2
2,244316,,,1
3,244324,,1,
4,244325,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 18 2022 8:30AM,244413,10,ISDIN Corporation
5,Aug 18 2022 8:30AM,244425,10,Emerginnova
10,Aug 18 2022 8:30AM,244428,10,"Methapharm, Inc."
11,Aug 18 2022 8:18AM,244435,10,ISDIN Corporation
29,Aug 18 2022 8:17AM,244434,10,Beach Products Inc
35,Aug 18 2022 8:09AM,244433,10,Emerginnova
37,Aug 18 2022 7:42AM,244432,19,"AdvaGen Pharma, Ltd"
46,Aug 18 2022 7:39AM,244431,19,ACG North America LLC
47,Aug 18 2022 7:33AM,244430,16,"Emersa Waterbox, LLC"
48,Aug 17 2022 5:56PM,244429,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 18 2022 8:30AM,244413,10,ISDIN Corporation,,,5
1,Aug 18 2022 8:30AM,244425,10,Emerginnova,1,,4
2,Aug 18 2022 8:30AM,244428,10,"Methapharm, Inc.",,,1
3,Aug 18 2022 8:18AM,244435,10,ISDIN Corporation,,,18
4,Aug 18 2022 8:17AM,244434,10,Beach Products Inc,,,6
5,Aug 18 2022 8:09AM,244433,10,Emerginnova,,,2
6,Aug 18 2022 7:42AM,244432,19,"AdvaGen Pharma, Ltd",,,9
7,Aug 18 2022 7:39AM,244431,19,ACG North America LLC,,,1
8,Aug 18 2022 7:33AM,244430,16,"Emersa Waterbox, LLC",,,1
9,Aug 17 2022 5:56PM,244429,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 8:30AM,244413,10,ISDIN Corporation,5,,
1,Aug 18 2022 8:30AM,244425,10,Emerginnova,4,,1
2,Aug 18 2022 8:30AM,244428,10,"Methapharm, Inc.",1,,
3,Aug 18 2022 8:18AM,244435,10,ISDIN Corporation,18,,
4,Aug 18 2022 8:17AM,244434,10,Beach Products Inc,6,,
5,Aug 18 2022 8:09AM,244433,10,Emerginnova,2,,
6,Aug 18 2022 7:42AM,244432,19,"AdvaGen Pharma, Ltd",9,,
7,Aug 18 2022 7:39AM,244431,19,ACG North America LLC,1,,
8,Aug 18 2022 7:33AM,244430,16,"Emersa Waterbox, LLC",1,,
9,Aug 17 2022 5:56PM,244429,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 8:30AM,244413,10,ISDIN Corporation,5,,
1,Aug 18 2022 8:30AM,244425,10,Emerginnova,4,,1
2,Aug 18 2022 8:30AM,244428,10,"Methapharm, Inc.",1,,
3,Aug 18 2022 8:18AM,244435,10,ISDIN Corporation,18,,
4,Aug 18 2022 8:17AM,244434,10,Beach Products Inc,6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 8:30AM,244413,10,ISDIN Corporation,5.0,NaN,NaN
1,Aug 18 2022 8:30AM,244425,10,Emerginnova,4.0,NaN,1.0
2,Aug 18 2022 8:30AM,244428,10,"Methapharm, Inc.",1.0,NaN,NaN
3,Aug 18 2022 8:18AM,244435,10,ISDIN Corporation,18.0,NaN,NaN
4,Aug 18 2022 8:17AM,244434,10,Beach Products Inc,6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 8:30AM,244413,10,ISDIN Corporation,5.0,0.0,0.0
1,Aug 18 2022 8:30AM,244425,10,Emerginnova,4.0,0.0,1.0
2,Aug 18 2022 8:30AM,244428,10,"Methapharm, Inc.",1.0,0.0,0.0
3,Aug 18 2022 8:18AM,244435,10,ISDIN Corporation,18.0,0.0,0.0
4,Aug 18 2022 8:17AM,244434,10,Beach Products Inc,6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4154477,154.0,3.0,13.0
12,488813,0.0,2.0,0.0
15,244355,2.0,0.0,23.0
16,733266,21.0,0.0,0.0
17,244406,1.0,0.0,0.0
19,1222080,32.0,9.0,10.0
20,1221752,107.0,12.0,5.0
21,1710925,7.0,0.0,0.0
22,733133,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4154477,154.0,3.0,13.0
1,12,488813,0.0,2.0,0.0
2,15,244355,2.0,0.0,23.0
3,16,733266,21.0,0.0,0.0
4,17,244406,1.0,0.0,0.0
5,19,1222080,32.0,9.0,10.0
6,20,1221752,107.0,12.0,5.0
7,21,1710925,7.0,0.0,0.0
8,22,733133,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,154.0,3.0,13.0
1,12,0.0,2.0,0.0
2,15,2.0,0.0,23.0
3,16,21.0,0.0,0.0
4,17,1.0,0.0,0.0
5,19,32.0,9.0,10.0
6,20,107.0,12.0,5.0
7,21,7.0,0.0,0.0
8,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,154.0
1,12,Released,0.0
2,15,Released,2.0
3,16,Released,21.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,19,20,21,22
Status,,,,,,,,,
Completed,13.0,0.0,23.0,0.0,0.0,10.0,5.0,0.0,0.0
Executing,3.0,2.0,0.0,0.0,0.0,9.0,12.0,0.0,0.0
Released,154.0,0.0,2.0,21.0,1.0,32.0,107.0,7.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,19,20,21,22
0,Completed,13.0,0.0,23.0,0.0,0.0,10.0,5.0,0.0,0.0
1,Executing,3.0,2.0,0.0,0.0,0.0,9.0,12.0,0.0,0.0
2,Released,154.0,0.0,2.0,21.0,1.0,32.0,107.0,7.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,19,20,21,22
0,Completed,13.0,0.0,23.0,0.0,0.0,10.0,5.0,0.0,0.0
1,Executing,3.0,2.0,0.0,0.0,0.0,9.0,12.0,0.0,0.0
2,Released,154.0,0.0,2.0,21.0,1.0,32.0,107.0,7.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()